In [ ]:
# Disease Symptom Prediction using LLM
# Import necessary libraries

import pandas as pd
import streamlit as st
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split #StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
import openai
import numpy as np
import joblib
import pickle
from sentence_transformers import SentenceTransformer

c:\Users\DELL\LLM-Disease_symptom_prediction\ChhirolyaTech_AI_ML_Intern_Candidates_Assignments\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
data=pd.read_csv("C:\\Users\\DELL\\ChhirolyaTech_AI_ML_Intern_Candidates_Assignments\\sanjay_ravichander_disease_symptom_predictor_LLM\\src\\data\\dataset\\dataset.csv")
sym_desc=pd.read_csv("C:\\Users\\DELL\\ChhirolyaTech_AI_ML_Intern_Candidates_Assignments\\sanjay_ravichander_disease_symptom_predictor_LLM\\src\\data\\dataset\\symptom_Description.csv")
sym_pre=pd.read_csv("C:\\Users\\DELL\\ChhirolyaTech_AI_ML_Intern_Candidates_Assignments\\sanjay_ravichander_disease_symptom_predictor_LLM\\src\\data\\dataset\\symptom_precaution.csv")
sym_sev=pd.read_csv("C:\\Users\\DELL\\ChhirolyaTech_AI_ML_Intern_Candidates_Assignments\\sanjay_ravichander_disease_symptom_predictor_LLM\\src\\data\\dataset\\Symptom-severity.csv")

In [4]:
print(f"The shape of data : {data.shape}")
print(f"The shape of sym_desc: {sym_desc.shape}")
print(f"The shape of sym_pre: {sym_pre.shape}")
print(f"The shape of sym_sev: {sym_sev.shape}")

The shape of data : (4920, 18)
The shape of sym_desc: (41, 2)
The shape of sym_pre: (41, 5)
The shape of sym_sev: (133, 2)


In [5]:
data.head(2)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.isnull().sum() / len(data) * 100

Disease        0.000000
Symptom_1      0.000000
Symptom_2      0.000000
Symptom_3      0.000000
Symptom_4      7.073171
Symptom_5     24.512195
Symptom_6     40.365854
Symptom_7     53.902439
Symptom_8     60.487805
Symptom_9     65.609756
Symptom_10    69.268293
Symptom_11    75.731707
Symptom_12    84.878049
Symptom_13    89.756098
Symptom_14    93.780488
Symptom_15    95.121951
Symptom_16    96.097561
Symptom_17    98.536585
dtype: float64

In [7]:
# Select the first 5 columns of the DataFrame
data = data.iloc[:, 0:6]

In [8]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN


In [9]:
sym_desc.head(2)

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...


In [11]:
sym_desc.isnull().sum() / len(sym_desc) * 100

Disease        0.0
Description    0.0
dtype: float64

In [12]:
sym_pre.head(2)

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out


In [13]:
sym_pre.isnull().sum() / len(sym_pre) * 100

Disease         0.000000
Precaution_1    0.000000
Precaution_2    0.000000
Precaution_3    2.439024
Precaution_4    2.439024
dtype: float64

In [14]:
sym_sev.head(2)

,Symptom,weight
0,itching,1
1,skin_rash,3


In [15]:
sym_sev.isnull().sum() / len(sym_sev) * 100

Symptom    0.0
weight     0.0
dtype: float64

In [16]:
# Convert all disease and symptom names to lowercase and remove leading/trailing spaces
data['Disease'] = data['Disease'].str.lower().str.strip()
sym_desc['Disease'] = sym_desc['Disease'].str.lower().str.strip()
sym_pre['Disease'] = sym_pre['Disease'].str.lower().str.strip()
sym_sev['Symptom'] = sym_sev['Symptom'].str.lower().str.strip()

In [17]:
# Fill NaNs in symptom columns with 'none'
data = data.fillna('none')

In [18]:
# Merge data with symptom descriptions
merged_data = pd.merge(data, sym_desc, on='Disease', how='inner')

# Merge the result with symptom precautions
df = pd.merge(merged_data, sym_pre, on='Disease', how='inner')

In [19]:
df.to_csv('C:/Users/DELL/Downloads/Final_Disease_Symptom_dataset.csv', index=False)

In [20]:
df.shape

(4800, 11)

In [21]:
df.isnull().sum() / len(df) * 100

Disease         0.0
Symptom_1       0.0
Symptom_2       0.0
Symptom_3       0.0
Symptom_4       0.0
Symptom_5       0.0
Description     0.0
Precaution_1    0.0
Precaution_2    0.0
Precaution_3    2.5
Precaution_4    2.5
dtype: float64

In [22]:
df['Description']

0       In humans, fungal infections occur when an inv...
1       In humans, fungal infections occur when an inv...
2       In humans, fungal infections occur when an inv...
3       In humans, fungal infections occur when an inv...
4       In humans, fungal infections occur when an inv...
                              ...                        
4795    Benign paroxysmal positional vertigo (BPPV) is...
4796    Acne vulgaris is the formation of comedones, p...
4797    Urinary tract infection: An infection of the k...
4798    Psoriasis is a common skin disorder that forms...
4799    Impetigo (im-puh-TIE-go) is a common and highl...
Name: Description, Length: 4800, dtype: object

In [23]:
# Preprocessing 
symptoms = df[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5']]
disease = df['Disease']

In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(symptoms,disease, test_size=0.2, random_state=42)


In [25]:
# %% Encode the symptoms only for training and testing set
label_encoders = {}
symptoms_encoded_train = pd.DataFrame()
symptoms_encoded_test = pd.DataFrame()

for col in X_train.columns:
    le = LabelEncoder()
    symptoms_encoded_train[col] = le.fit_transform(X_train[col])
    symptoms_encoded_test[col] = le.transform(X_test[col])
    label_encoders[col] = le

In [26]:
# Create embeddings for the descriptions
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Need to merge or map descriptions to symptoms
# Need to get descriptions based on the diseases
description_map = dict(zip(sym_desc['Disease'], sym_desc['Description']))

# Generating description embeddings based on the y_train and y_test
description_embeddings_train = []
description_embeddings_test = []

for disease in y_train:
    description_embeddings_train.append(description_map.get(disease, ""))
for disease in y_test:
    description_embeddings_test.append(description_map.get(disease, ""))

# Encoding the descriptions into embeddings
description_embeddings_train = sentence_model.encode(description_embeddings_train)
description_embeddings_test = sentence_model.encode(description_embeddings_test)

# Combining symptom features with description embeddings
X_combined_train = np.hstack((symptoms_encoded_train.values, description_embeddings_train))
X_combined_test = np.hstack((symptoms_encoded_test.values, description_embeddings_test))


In [ ]:
# Defining hyperparameter grid
#param_grid = {
    #'n_estimators': [50, 100, 200],
    #'max_features': ['sqrt', 'log2', None],
    #'max_depth': [None, 10, 20, 30],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4]
#}

In [27]:
# grid_search = GridSearchCV(estimator=RandomForestClassifier(),
                           #param_grid=param_grid,
                           #scoring='accuracy',
                           #cv=2,
                           #verbose=2,
                           #n_jobs=-1)
# grid_search.fit(X_combined_train, y_train)

# Fit the model
model_rf=RandomForestClassifier()
model_rf.fit(X_combined_train, y_train)

RandomForestClassifier()

In [28]:
# %% Evaluate the best model
#best_model = grid_search.best_estimator_
y_pred = model_rf.predict(X_combined_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

1.0
                                         precision    recall  f1-score   support

(vertigo) paroymsal  positional vertigo       1.00      1.00      1.00        25
                                   acne       1.00      1.00      1.00        27
                                   aids       1.00      1.00      1.00        21
                    alcoholic hepatitis       1.00      1.00      1.00        20
                                allergy       1.00      1.00      1.00        35
                              arthritis       1.00      1.00      1.00        18
                       bronchial asthma       1.00      1.00      1.00        28
                   cervical spondylosis       1.00      1.00      1.00        21
                            chicken pox       1.00      1.00      1.00        22
                    chronic cholestasis       1.00      1.00      1.00        19
                            common cold       1.00      1.00      1.00        18
                       

In [29]:
# Save the best model
joblib.dump(model_rf, 'C:\\Users\\DELL\\Downloads\\best_rf_model.pkl')

['C:\\Users\\DELL\\Downloads\\best_rf_model.pkl']

In [30]:
# Save the Sentence Transformer model
pickle.dump(sentence_model, open('C:\\Users\\DELL\\Downloads\\sentence_transformer.pkl', 'wb'))

In [ ]:
# Retrieve the best parameters and score
# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"Best Cross-Validation Score: {grid_search.best_score_:.2f}")

In [31]:
def get_llm_response(symptoms, predicted_disease):
    prompt = (
        f"The user has the following symptoms: {', '.join(symptoms)}. Based on this, the predicted disease is '{predicted_disease}'. "
        "What are the recommended precautions, remedies, and common over-the-counter medications for this condition? "
        "Please list medications that are generally considered safe for adults, unless otherwise specified, and provide any necessary precautions."
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a knowledgeable medical assistant providing general advice."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content']
